# 6 Resampling Method and Model Selection


## Cross Validation


### Leave-One-Out-Cross-Validation (LOOCV)

For $n$ samples, we fit models on $(n-1)$ samples while validating on the last sample. Repeat this for each sample for $n$ times. Compute the average loss with 
$${\rm CV} = \frac 1n\sum_{i=1}^n (y_i - f_{(i)}(x_i))^2$$
where $f_{(i)}$ is the model fitted on samples excluding $(x_i,y_i)$.


<br>


It is often expensive to compute LOOCV as it needs to fit the model for $n$ times. However, for linear regression, LOOCV has an easier form as below. [Ref](https://stats.stackexchange.com/questions/164223/proof-of-loocv-formula)

**Theorem** For linear regression given covariates $X\in\mathbb R^{n\times(k+1)}$ and response $y\in\mathbb R^n$, assume any $(n-1)\times (k+1)$  submatrix is of full rank, then it has LOOCV given below.
$${\rm CV} = \frac1n \sum_{i=1}^n \left(\frac{y_i - \hat{y_i}}{1 - h_{ii}}\right)^2$$
where $\hat y_i$ is the prediction with the model fitted on the $n$ data, while $h_{ii}$ is the $(i,i)$ entry of $X(X^TX)^{-1}X$.

**Proof** Assume $X = [x_1,\dotsc,x_n]^T$. For each $i$, we remove row $i$ of $X,y$ to obtain design matrix $X_{(i)}$ and response $y_{(i)}$. The coefficient $\hat\beta_{(i)}$ is given by 
$$ (X_{(i)}^TX_{(i)})^{-1}X_{(i)}^Ty_{(i)}
=(X_{(i)}^TX_{(i)})^{-1}(X^Ty - x_iy_i).
$$

Observe that $X_{(i)}^TX_{(i)} = X^TX - x_ix_i^T$, by Sherman-Morrison formula we have
$$(X_{(i)}^TX_{(i)})^{-1}=(X^TX)^{-1} +\frac{(X^TX)^{-1} x_ix_i^T(X^TX)^{-1} }{1-x_i^T(X^TX)^{-1} x_i}
=(X^TX)^{-1} +\frac{(X^TX)^{-1} x_ix_i^T(X^TX)^{-1} }{1-h_{ii}}.$$

If we write $\hat \beta = (X^TX)^{-1}X^Ty$ and $\hat y  =X\hat\beta$, then 
$$\begin{aligned}
\hat\beta_{(i)}&= (X_{(i)}^TX_{(i)})^{-1}(X^Ty - x_iy_i)\\ 
&= \left((X^TX)^{-1}  +\frac{(X^TX)^{-1} x_ix_i^T(X^TX)^{-1} }{1-h_{ii}}\right)(X^Ty - x_iy_i)\\ 
&= \hat \beta +\frac{(X^TX)^{-1} x_ix_i^T\hat\beta }{1-h_{ii}}-(X^TX)^{-1}x_iy_i -\frac{(X^TX)^{-1} x_ix_i^T(X^TX)^{-1} x_iy_i}{1-h_{ii}}
\\ &= \hat \beta +\frac{(X^TX)^{-1} x_ix_i^T\hat\beta }{1-h_{ii}}-\frac{(X^TX)^{-1} x_iy_i}{1-h_{ii}}
\\ &= \hat \beta -\frac{(X^TX)^{-1} x_i(y_i-\hat y_i) }{1-h_{ii}}
\end{aligned}
$$

Validate the model $\hat\beta_{(i)}$ on $(x_i,y_i)$ gives the local loss, 
$$\hat\epsilon_i  = y_i - x_i^T\hat\beta_{(i)} = y_i -x_i^T\hat\beta + \frac{x_i^T(X^TX)^{-1} x_i(y_i-\hat y_i) }{1-h_{ii}} =\frac{y_i- \hat y_i}{1 - h_{ii}}.$$

Sum the squares up yields the desired result.

In [73]:
import numpy as np
np.random.seed(0)
N = 5
X = np.random.randn(N,3)
H = X @ np.linalg.inv(X.T @ X) @ X.T
h = H.flat[::N+1]
y = np.random.randn(N)
beta = np.linalg.inv(X.T @ X) @ X.T @ y
v1 = (y - X @ beta) / (1 - h)
v2 = []
for i in range(N):
    Xi = X[1:] if i == 0 else (X[:-1] if i == N-1 else np.vstack((X[:i], X[i+1:])))
    yi = y[1:] if i == 0 else (y[:-1] if i == N-1 else np.concatenate((y[:i], y[i+1:])))
    betai = np.linalg.solve(Xi.T @ Xi, Xi.T @ yi)
    v2.append(y[i] - np.dot(betai, X[i]))

# different (but equivalent) computations of loss
print('Loss1 = %s\nLoss2 = %s'%(v1, np.array(v2)))

Loss1 = [ 0.49419552  0.74290667  0.5209724   0.44593625 -0.96498697]
Loss2 = [ 0.49419552  0.74290667  0.5209724   0.44593625 -0.96498697]


### K-Fold Cross Validation

Still it is often the case that LOOCV is too expensive. We can use the K-fold cross validation. This is done by spliting the data into $k$ parts. We fit the model with $(k-1)$ parts and validate on the last part. Perform it on each of the parts for $k$ times. Average the validation loss.

## Jackknife and Bootstrap

If we have an estimator $T$ for some model parameter and now we want to estimate the standard deviation / variance of $T$. (It is useful in hypothesis testing and estimating confidence interval.) Then we can apply the method of Jackknife or Bootstrap.

### Jackknife

Given i.i.d. sample $X_1,\dotsc,X_n$ and let $T(X_1,\dotsc,X_n)$ be a function that produces an estimator. Now we remove $X_1$ from the sample and the compute the estimator with the remaining samples to attain $T_1$: $T_1 = T(X_2,\dotsc,X_n)$. Similarly we can obtain $T_1,T_2,\dotsc,T_n$ by removing $X_i$ and computing with the remaining $(n-1)$ samples. Let $\bar T = \frac 1n \sum_{i=1}^n T_i$, then we estimate the variance of $T$ by 

$$\widehat{{\rm SE}(T)_{\rm jack}}^2 = \frac{n-1}{n}\sum_{i=1}^n (T_i - \bar T)^2.$$

This is called the Jackknife estimator.

### Bootstrap

Given i.i.d. sample $X_1,\dotsc,X_n$ and let $T(X_1,\dotsc,X_n)$ be a function that produces an estimator. Randomly get $n$ samples with replacement from the sample and compute an estimation $T_1$. Repeat it with $B$ times to obtain $T_1,T_2,\dotsc,T_B$. Let $\bar T = \frac 1n \sum_{i=1}^B T_i$, then we estimate the variance of $T$ by 

$$\widehat{{\rm SE}(T)_{\rm boot}}^2 = \frac{1}{B-1}\sum_{i=1}^n (T_i - \bar T)^2.$$

This is called the Bootstrap estimator.



## Subset Selection

### Forward / Backward Selection

### Hybrid Approach 

Combine forward and backward selection in the process.


## Choosing Optimal Model


### Generalization Gap / Covariance Penalty

Given data $X\in\mathbb R^{n\times k}$ and response $y\in\mathbb R$ with underlying relation $y = f(X)+\epsilon$ where $\epsilon_i$ are i.i.d. noise with zero mean and variance $\sigma^2$, we can fit a model $\hat f$ such that $\hat y = \hat f(X)$. And we can compute the average loss on the training data,
$$\mathcal L=\frac 1n\Vert y - \hat f(X) \Vert^2= \frac 1n\sum_{i=1}^n \Vert f(X)+\epsilon - \hat f(X)\Vert^2.$$

However, when fixing $X$, the training loss $\mathcal L$ depends on the response $y$ we observe, or $\epsilon$. The loss $\mathcal L$ we observe might be deceive since we might have accidentally fix on observations with small noise. If we want to compute the expectance of our mean square error of $\hat f$ with **covariates** $X$ **fixed** while $\epsilon$ is randomly generated, we can show that 
$$ \mathcal L'(\hat f,X) = \frac 1n\sum_{i=1}^n \Vert y' - \hat f(X)\Vert^2= \frac 1n\sum_{i=1}^n \Vert f(X)+\epsilon' - \hat f(X)\Vert^2
$$

**Theorem**
$$\mathbb E(\mathcal L')=\mathbb E(\mathcal L)+\frac 2n{\rm tr}\left\{{\rm Cov}(\hat f(X),f(X))\right\}.$$

In other words, even on the same data $X$, training loss is often underestimated due to "overfitting".

<br>

**Proof** Take the expectance of the two definitions of $\mathcal L$ and $\mathcal L'$, we have 
$$\begin{aligned}n\mathbb E(\mathcal L) &= \mathbb E\left\{\Vert (f(X) -\hat f(X)) + \epsilon\Vert^2\right\}
= \mathbb E\left\{\Vert f(X) -\hat f(X)\Vert^2\right\}+n\sigma^2 + 2\mathbb E\left\{(f(X) -\hat f(X))^T\epsilon\right\}
\end{aligned}
$$
$$\begin{aligned}n\mathbb E(\mathcal L') &= \mathbb E\left\{\Vert (f(X) -\hat f(X)) + \epsilon'\Vert^2\right\}
= \mathbb E\left\{\Vert f(X) -\hat f(X)\Vert^2\right\}+n\sigma^2 + 2\mathbb E\left\{(f(X) -\hat f(X))^T\epsilon'\right\}
\end{aligned}
$$

The only difference is that, $\hat f(X)$ is correlated with $\epsilon$, while independent with $\epsilon'$. So we can eliminate the term $\mathbb E\left\{(f(X) -\hat f(X))^T\epsilon'\right\}$ to conclude
$$\mathbb E(\mathcal L')=\mathbb E(\mathcal L)- \frac 2n\mathbb E\left\{(f(X) -\hat f(X))^T\epsilon\right\}=\mathbb E(\mathcal L)+\frac 2n{\rm tr}\left\{{\rm Cov}(\hat f(X),f(X))\right\}.$$


#### Linear Regression

In the special case of linear regresssion where $\hat y = \hat f(X) = X(X^TX)^{-1}X^Ty$ and $X\in\mathbb R^{n\times k}$, the covariance penalty term is given by
$$\frac 2n{\rm tr}\left\{{\rm Cov}(\hat f(X),f(X))\right\}=\frac 2n{\rm tr}\left\{{\rm Cov}(X(X^TX)^{-1}X^Ty, y)\right\}
=\frac 2n\sigma^2 {\rm tr}(X(X^TX)^{-1}X^T)=\frac 2n\sigma^2k.$$

### Degree of Freedom 

We can use more generalized method to define the degree of freedom.

$${\rm df} = \sigma^{-2}{\rm tr}\left\{{\rm Cov}(\hat f(X),f(X))\right\}.$$

Note that the caculation of linear regresssion above indicates that it is reasonable.